In [3]:
%load_ext autoreload
%autoreload 2

import recommender_utils
import math
import copy

import model_CNN as model_cnn
import torch
# from torch.nn import functional as F 
import numpy as np


/Users/user/Documents/Nauka/Studia/Magisterskie/Magisterka/Recommender/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
movie_df, movie_feature_headers, num_feature_headers = recommender_utils.get_movies_data(filepath='data_small/movies.csv', separator=r',', movies_columns_to_drop=['genres'], only_genres=False)

data_train = recommender_utils.get_ratings_data(filepath='data_small/train.csv', separator=r',', dtypes=recommender_utils.dtypes)
data_val = recommender_utils.get_ratings_data(filepath='data_small/validate.csv', separator=r',', dtypes=recommender_utils.dtypes)
data_test = recommender_utils.get_ratings_data(filepath='data_small/test.csv', separator=r',', dtypes=recommender_utils.dtypes)

(num_users_train, num_movies_train), (X_train, y_train), _ = recommender_utils.create_dataset_cnn(data_train)
(num_users_val, num_movies_val), (X_val, y_val), _ = recommender_utils.create_dataset_cnn(data_val)
(num_users_test, num_movies_test), (X_test, y_test), _ = recommender_utils.create_dataset_cnn(data_test)

print("Dataset treningowy:")
print(f"\tEmbeddings: {num_users_train} uzytkowników, {num_movies_train} filmów")
print(f"\tX wymiar: {X_train.shape}")
print(f"\tY shape: {y_train.shape}")
print(X_train.user_id.max())
print(X_train.movie_id.max())

print("Dataset walidacyjny:")
print(f"\tEmbeddings: {num_users_val} uzytkowników, {num_movies_val} filmów")
print(f"\tX wymiar: {X_val.shape}")
print(f"\tY shape: {y_val.shape}")
print(X_val.user_id.max())
print(X_val.movie_id.max())

print("Dataset testowy:")
print(f"\tEmbeddings: {num_users_test} uzytkowników, {num_movies_test} filmów")
print(f"\tX wymiar: {X_test.shape}")
print(f"\tY shape: {y_test.shape}")
print(X_test.user_id.max())
print(X_test.movie_id.max())

datasets = {'train': (X_train, y_train), 'val': (X_val, y_val)}
dataset_sizes = {'train': len(X_train), 'val': len(X_val)}

minmax = y_train.min().astype(float), y_train.max().astype(float)

num_users_all = num_users_train + num_users_val + num_users_test
num_movies_all = num_movies_train + num_movies_val + num_movies_test


Dataset treningowy:
	Embeddings: 427 uzytkowników, 8500 filmów
	X wymiar: (68495, 2)
	Y shape: (68495,)
426
8499
Dataset walidacyjny:
	Embeddings: 61 uzytkowników, 5536 filmów
	X wymiar: (17280, 2)
	Y shape: (17280,)
60
5535
Dataset testowy:
	Embeddings: 122 uzytkowników, 3975 filmów
	X wymiar: (15061, 2)
	Y shape: (15061,)
121
3974


In [5]:
model = model_cnn.EmbeddingNet(
    n_users=num_users_all, n_movies=num_movies_all, 
    n_factors=150, hidden=[500, 500, 500], 
    embedding_dropout=0.05, dropouts=[0.5, 0.5, 0.25])

In [6]:
RANDOM_STATE = 1
torch.manual_seed(RANDOM_STATE)
torch.cuda.manual_seed(RANDOM_STATE)
lr = 1e-6
wd = 1e-5
bs = 1000
n_epochs = 100
patience = 10
no_improvements = 0
best_loss = np.inf
best_weights = None
use_scheduler = False
history = []
lr_history = []

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model.to(device)
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
iterations_per_epoch = int(math.ceil(dataset_sizes['train'] // bs))
if use_scheduler:
    scheduler = model_cnn.CyclicLR(optimizer, model_cnn.cosine(t_max=iterations_per_epoch * 2, eta_min=lr/10))

for epoch in range(n_epochs):
    stats = {'epoch': epoch + 1, 'total': n_epochs}
    
    for phase in ('train', 'val'):
        training = phase == 'train'
        running_loss = 0.0
        n_batches = 0
        
        for batch in model_cnn.batches(*datasets[phase], shuffle=training, bs=bs):
            x_batch, y_batch = [b.to(device) for b in batch]
            optimizer.zero_grad()
        
            # compute gradients only during 'train' phase
            with torch.set_grad_enabled(training):
                outputs = model(x_batch[:, 0], x_batch[:, 1], minmax)
                loss = criterion(outputs, y_batch)
                
                # don't update weights and rates when in 'val' phase
                if training:
                    if use_scheduler:
                        scheduler.step()
                    loss.backward()
                    optimizer.step()
                    if use_scheduler:
                        lr_history.extend(scheduler.get_lr())
                    
            running_loss += loss.item()
            
        epoch_loss = running_loss / dataset_sizes[phase]
        stats[phase] = epoch_loss
        
        # early stopping: save weights of the best model so far
        if phase == 'val':
            if epoch_loss < best_loss:
                print('loss improvement on epoch: %d' % (epoch + 1))
                best_loss = epoch_loss
                best_weights = copy.deepcopy(model.state_dict())
                no_improvements = 0
            else:
                no_improvements += 1
                
    history.append(stats)
    print('[{epoch:03d}/{total:03d}] train: {train:.4f} - val: {val:.4f}'.format(**stats))
    if no_improvements >= patience:
        print('early stopping after epoch {epoch:03d}'.format(**stats))
        break

loss improvement on epoch: 1
[001/100] train: 1.6058 - val: 1.3672
loss improvement on epoch: 2
[002/100] train: 1.5715 - val: 1.3357
loss improvement on epoch: 3
[003/100] train: 1.5383 - val: 1.3037
loss improvement on epoch: 4
[004/100] train: 1.5068 - val: 1.2793
loss improvement on epoch: 5
[005/100] train: 1.4802 - val: 1.2517


KeyboardInterrupt: 

In [131]:
torch.save(model.state_dict(), "models/cnn_100epochs_wo_features_base.pt")
#epochs_80_with_20_features = loss_through_epochs

In [ ]:
print(lr_history)